Consider the following parameterization of a policy: There is a \preference" for each
action, for each column and for each row. Thus the set of preferences can be denoted by
θx(N; 0); θy(N; 0); θx(S; 0); θy(S; 0); θx(E; 0); θy(E; 0); :::; θx(W; 9); θy(W; 9), for a total of
80 preference values. The total preference for an action a in a state (i, j) is given by
θx(a; i) + θy(a; j). The action probabilities are generated by a soft-max function using
these preferences.
4. Implement a MC policy gradient algorithm. Choose appropriate learning rates, and turn
in two curves for each variant as indicated in the first part as well as the optimal policies
learnt.

In [1]:
import gym
import gym_pdw

import numpy as np
from matplotlib import pyplot as plt

In [ ]:
def sftmax_action(state, parameters):
    x, y = state
    north = parameters[x,0,0] + parameters[y,1,0]
    east = parameters[x,0,1] + parameters[y,1,1]
    west = parameters[x,0,2] + parameters[y,1,2]
    south = parameters[x,0,3] + parameters[y,1,3]
    
    preferences = [north, east, west, south]
    action = np.random.choice([0,1,2,3], preferences)
    
    return action, preferences[action]

In [ ]:
def selected_param_pos(state, action):
    direction = action
    row,col = state
    return row, col, direction

In [ ]:
def Update(parameters, states, actions, rewards, params_pos, prob, gamma, alpha):
    
    # Initialize return
    G = 0
    
    i=0
    for prob, pos, reward in zip(actions[::-1], params_pos[::-1], rewards[::-1]):
        
        # The positions of parameters that was used to select the action in ith time step
        row, col, direction = pos
        
        # The parameters that were used in action selection
        theta1 = parameters[row, 0, direction]
        theta2 = parameters[col, 1, direction]
                
        # The return 
        G = reward + pow(gamma,i)*G
        
        # The update equations
        theta1 = theta1 + alpha*pow(gamma,i)*(1 - prob)
        theta2 = theta2 + alpha*pow(gamma,i)*(1 - prob)
        
        parameters[row, 0, direction] = theta1
        parameters[col, 1, direction] = theta2
        
        i+=1


In [ ]:

def MC_PG(alpha, gamma):
    
    # For each row,direction and For each column,direction one parameter
    parameters = np.ones([12,2,4])
    
    
    while True:
        # An episode
        
        # Store all the states, actions, rewards till the end of episode 
        states = []
        actions = []
        rewards = []
        
        # The parameters that are selected in each time step
        params_pos = []
        # The probability of picking those parameters
        probs = []
        
        # Reset environment
        env.reset()
        
        # Initialize state
        curr_state = env.get_state()
        
        while True:
            # For each step of an episode
            
            # select an action according to softmax probabilities
            action, prob = sftmax_action(curr_state, parameters)
            
            # Get reward and next state
            next_state, reward = env.step(action)
            
            probs.append(prob)
            params_pos.append(selected_param_pos(state, action))
            
            states.append(curr_state)
            actions.append(action)
            rewards.append(reward)

            # Increase the number of steps
            steps[episode] +=1
            
            # Average reward till now
            avg_reward[episode] = avg_reward[episode] + (reward - avg_reward[episode])/steps[episode]

            curr_state = next_state
            print(curr_state, "curr_state", env.start_positions)

            if curr_state == goal_pos:
                print("yes", steps[episode])
                break
        
        # Update the parameters used in action selection in the episode
        update(parameters, states, actions, rewards, params, probs, gamma, alpha)
            
    
    return avg_reward, steps
    

In [3]:
if __name__=='__main__':

    env = gym.make('gym_pdw:pdw-v0')
    
    
    